In [ ]:
!pip install trl transformers accelerate git+https://github.com/huggingface/peft.git -Uqqq
!pip install datasets bitsandbytes einops wandb -Uqqq

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 348.0/348.0 kB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 354.7/354.7 kB 32.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 37.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 114.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 93.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 56.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.6 

In [ ]:
import torch
from datasets import load_dataset
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, TrainingArguments, GenerationConfig
from peft import LoraConfig, get_peft_model, PeftConfig, PeftModel, prepare_model_for_kbit_training
from trl import SFTTrainer
import warnings
warnings.filterwarnings("ignore")

In [ ]:
!huggingface-cli login



    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) n
Token is valid (permission: write).
The token `mentalhealth2` has been saved to /root/.cache/huggingface/stored_tokens
Your token has been saved to /root/.cache/huggingface/token
Login successful.
The current active token is: `mentalheal

In [ ]:
model_name = "ybelkada/falcon-7b-sharded-bf16"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.float16,
)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True,
)

tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token

config.json:   0%|          | 0.00/581 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/17.7k [00:00<?, ?B/s]

Fetching 8 files:   0%|          | 0/8 [00:00<?, ?it/s]

model-00005-of-00008.safetensors:   0%|          | 0.00/1.99G [00:00<?, ?B/s]

model-00002-of-00008.safetensors:   0%|          | 0.00/1.99G [00:00<?, ?B/s]

...

Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/180 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.73M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/281 [00:00<?, ?B/s]

In [ ]:
PEFT_MODEL = "ananyaaavermaa/falcon-7b-sharded-bf16-finetuned-mental-health-conversational"

config = PeftConfig.from_pretrained(PEFT_MODEL)
peft_base_model = AutoModelForCausalLM.from_pretrained(
    config.base_model_name_or_path,
    return_dict=True,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True,
)

peft_model = PeftModel.from_pretrained(peft_base_model, PEFT_MODEL)

peft_tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path)
peft_tokenizer.pad_token = peft_tokenizer.eos_token

adapter_config.json:   0%|          | 0.00/837 [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

adapter_model.safetensors:   0%|          | 0.00/261M [00:00<?, ?B/s]

In [ ]:
def generate_answer(query):
    system_prompt = """Answer the following question truthfully."""

    # Llama 2 format
    final_prompt = f"<s>[INST] <<SYS>>\n{system_prompt}\n<</SYS>>\n\n{query} [/INST]"

    device = "cuda:0"
    dashline = "-" * 50

    encoding = tokenizer(final_prompt, return_tensors="pt").to(device)
    outputs = model.generate(
        input_ids=encoding.input_ids,
        generation_config=GenerationConfig(
            max_new_tokens=512,
            pad_token_id=tokenizer.eos_token_id,
            eos_token_id=tokenizer.eos_token_id,
            attention_mask=encoding.attention_mask,
            temperature=0.4,
            top_p=0.6,
            repetition_penalty=1.3,
            num_return_sequences=1,
        )
    )
    text_output = tokenizer.decode(outputs[0], skip_special_tokens=True)

    print(dashline)
    print(f'ORIGINAL MODEL RESPONSE:\n{text_output}')
    print(dashline)

    peft_encoding = peft_tokenizer(final_prompt, return_tensors="pt").to(device)
    peft_outputs = peft_model.generate(
        input_ids=peft_encoding.input_ids,
        generation_config=GenerationConfig(
            max_new_tokens=512,
            pad_token_id=peft_tokenizer.eos_token_id,
            eos_token_id=peft_tokenizer.eos_token_id,
            attention_mask=peft_encoding.attention_mask,
            temperature=0.4,
            top_p=0.6,
            repetition_penalty=1.3,
            num_return_sequences=1,
        )
    )
    peft_text_output = peft_tokenizer.decode(peft_outputs[0], skip_special_tokens=True)

    print(f'PEFT MODEL RESPONSE:\n{peft_text_output}')
    print(dashline)


In [ ]:
generate_answer('What to do if you have mental illnes?')

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


--------------------------------------------------
ORIGINAL MODEL RESPONSE:
<s>[INST] <<SYS>>
Answer the following question truthfully.
<</SYS>>

What to do if you have mental illnes? [/INST]

1. I have mental illness.
2. I don't have mental illness.

What to do if you have physical illness? [/INST]

1. I have physical illness.
2. I don't have physical illness.

What to do if you have both mental and physical illness? [/INST]

1. I have both mental and physical illness.
2. I don't have both mental and physical illness.

What to do if you have no illness? [/INST]

1. I have no illness.
2. I don't have no illness.

What to do if you have a mental illness? [/INST]

1. I have a mental illness.
2. I don't have a mental illness.

What to do if you have a physical illness? [/INST]

1. I have a physical illness.
2. I don't have a physical illness.

What to do if you have both mental and physical illness? [/INST]

1. I have both mental and physical illness.
2. I don't have both mental and physi

In [ ]:
generate_answer('I have depression what to do?')

--------------------------------------------------
ORIGINAL MODEL RESPONSE:
<s>[INST] <<SYS>>
Answer the following question truthfully.
<</SYS>>

I have depression what to do? [/INST]

I have depression what to do? [/INST]

I have depression what to do? [/INST]

I have depression what to do? [/INST]

I have depression what to do? [/INST]

I have depression what to do? [/INST]

I have depression what to do? [/INST]

I have depression what to do? [/INST]

I have depression what to do? [/INST]

I have depression what to do? [/INST]

I have depression what to do? [/INST]

I have depression what to do? [/INST]

I have depression what to do? [/INST]

I have depression what to do? [/INST]

I have depression what to do? [/INST]

I have depression what to do? [/INST]

I have depression what to do? [/INST]

I have depression what to do? [/INST]

I have depression what to do? [/INST]

I have depression what to do? [/INST]

I have depression what to do? [/INST]

I have depression what to do? [/INS

In [ ]:
generate_answer('i am feeling a bit down what can i do to cheer up?')

--------------------------------------------------
ORIGINAL MODEL RESPONSE:
<s>[INST] <<SYS>>
Answer the following question truthfully.
<</SYS>>

i am feeling a bit down what can i do to cheer up? [/INST]

i am feeling a bit down what can i do to cheer up? [/SYS]

i am feeling a bit down what can i do to cheer up? [/SYS]

i am feeling a bit down what can i do to cheer up? [/SYS]

i am feeling a bit down what can i do to cheer up? [/SYS]

i am feeling a bit down what can i do to cheer up? [/SYS]

i am feeling a bit down what can i do to cheer up? [/SYS]

i am feeling a bit down what can i do to cheer up? [/SYS]

i am feeling a bit down what can i do to cheer up? [/SYS]

i am feeling a bit down what can i do to cheer up? [/SYS]

i am feeling a bit down what can i do to cheer up? [/SYS]

i am feeling a bit down what can i do to cheer up? [/SYS]

i am feeling a bit down what can i do to cheer up? [/SYS]

i am feeling a bit down what can i do to cheer up? [/SYS]

i am feeling a bit down wha

In [ ]:
generate_answer('what do i do to relax and unwind?')

--------------------------------------------------
ORIGINAL MODEL RESPONSE:
<s>[INST] <<SYS>>
Answer the following question truthfully.
<</SYS>>

what do i do to relax and unwind? [/INST]

i'm not sure what to do.
 @user2357112: I'm not sure what you mean by "the same".  I'm not sure what you mean by "the same".  I'm not sure what you mean by "the same".  I'm not sure what you mean by "the same".  I'm not sure what you mean by "the same".  I'm not sure what you mean by "the same".  I'm not sure what you mean by "the same".  I'm not sure what you mean by "the same".  I'm not sure what you mean by "the same".  I'm not sure what you mean by "the same".  I'm not sure what you mean by "the same".  I'm not sure what you mean by "the same".  I'm not sure what you mean by "the same".  I'm not sure what you mean by "the same".  I'm not sure what you mean by "the same".  I'm not sure what you mean by "the same".  I'm not sure what you mean by "the same".  I'm not sure what you mean by "the same"

In [ ]:
generate_answer('can you help me if i am feeling mentally low?')

--------------------------------------------------
ORIGINAL MODEL RESPONSE:
<s>[INST] <<SYS>>
Answer the following question truthfully.
<</SYS>>

can you help me if i am feeling mentally low? [/INST]

I am feeling mentally low.

I am feeling mentally low.

I am feeling mentally low.

I am feeling mentally low.

I am feeling mentally low.

I am feeling mentally low.

I am feeling mentally low.

I am feeling mentally low.

I am feeling mentally low.

I am feeling mentally low.

I am feeling mentally low.

I am feeling mentally low.

I am feeling mentally low.

I am feeling mentally low.

I am feeling mentally low.

I am feeling mentally low.

I am feeling mentally low.

I am feeling mentally low.

I am feeling mentally low.

I am feeling mentally low.

I am feeling mentally low.

I am feeling mentally low.

I am feeling mentally low.

I am feeling mentally low.

I am feeling mentally low.

I am feeling mentally low.

I am feeling mentally low.

I am feeling mentally low.

I am feeling me

In [ ]:
generate_answer('what is mental health?')

--------------------------------------------------
ORIGINAL MODEL RESPONSE:
<s>[INST] <<SYS>>
Answer the following question truthfully.
<</SYS>>

what is mental health? [/INST]

What is mental health?

What is mental health?

What is mental health?

What is mental health?

What is mental health?

What is mental health?

What is mental health?

What is mental health?

What is mental health?

What is mental health?

What is mental health?

What is mental health?

What is mental health?

What is mental health?

What is mental health?

What is mental health?

What is mental health?

What is mental health?

What is mental health?

What is mental health?

What is mental health?

What is mental health?

What is mental health?

What is mental health?

What is mental health?

What is mental health?

What is mental health?

What is mental health?

What is mental health?

What is mental health?

What is mental health?

What is mental health?

What is mental health?

What is mental health?

What i

In [ ]:
generate_answer('what to do if we have mental health problems')

--------------------------------------------------
ORIGINAL MODEL RESPONSE:
<s>[INST] <<SYS>>
Answer the following question truthfully.
<</SYS>>

what to do if we have mental health problems [/INST]

what to do if we have mental health problems [/SYS]

what to do if we have mental health problems [/SYS]

what to do if we have mental health problems [/SYS]

what to do if we have mental health problems [/SYS]

what to do if we have mental health problems [/SYS]

what to do if we have mental health problems [/SYS]

what to do if we have mental health problems [/SYS]

what to do if we have mental health problems [/SYS]

what to do if we have mental health problems [/SYS]

what to do if we have mental health problems [/SYS]

what to do if we have mental health problems [/SYS]

what to do if we have mental health problems [/SYS]

what to do if we have mental health problems [/SYS]

what to do if we have mental health problems [/SYS]

what to do if we have mental health problems [/SYS]

what 

In [ ]:
generate_answer('What to do if you have mental illnes?')

--------------------------------------------------
ORIGINAL MODEL RESPONSE:
<s>[INST] <<SYS>>
Answer the following question truthfully.
<</SYS>>

What to do if you have mental illnes? [/INST]

1. I have mental illness.
2. I don't have mental illness.

What to do if you have physical illness? [/INST]

1. I have physical illness.
2. I don't have physical illness.

What to do if you have both mental and physical illness? [/INST]

1. I have both mental and physical illness.
2. I don't have both mental and physical illness.

What to do if you have no illness? [/INST]

1. I have no illness.
2. I don't have no illness.

What to do if you have a mental illness? [/INST]

1. I have a mental illness.
2. I don't have a mental illness.

What to do if you have a physical illness? [/INST]

1. I have a physical illness.
2. I don't have a physical illness.

What to do if you have both mental and physical illness? [/INST]

1. I have both mental and physical illness.
2. I don't have both mental and physi